In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [7]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.874645,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.890157,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.906034,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.919415,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.399834,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.832198,0.050588,1.519517,1.163052,-0.960847,-0.093961,0.627995
158,1.402752,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.824048,0.042723,1.476375,1.168317,-0.968515,-0.074705,0.638908
159,1.405671,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.815727,0.029152,1.433232,1.173583,-0.976183,-0.055448,0.649820
160,1.408589,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.810089,-0.007439,1.390090,1.178848,-0.983851,-0.036192,0.660733


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
157    338.767
158    419.962
159    387.800
160    391.122
161    409.739
Name: Minas Gerais - Consumo de Cimento (t), Length: 162, dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
162,1.414426,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,0.798904,0.018612,1.303806,1.189379,-0.999187,0.002321,0.682559
163,1.417345,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,0.799394,0.033397,1.260664,1.194644,-1.006855,0.021577,0.693472
164,1.420263,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,0.797051,0.046888,1.217522,1.199909,-1.014523,0.040833,0.704385
165,1.423182,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,0.798229,0.060840,1.174379,1.205174,-1.022191,0.060089,0.715298
166,1.426100,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,0.797215,0.064701,1.131237,1.210440,-1.029859,0.079346,0.726211
167,1.429018,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,0.796730,0.073529,1.088095,1.215705,-1.037527,0.098602,0.737124
168,1.431937,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,0.796755,0.083989,1.044953,1.220970,-1.045195,0.117858,0.748037
169,1.425998,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,0.797325,0.093382,1.026626,1.220196,-1.031368,0.129561,0.749541
170,1.420059,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.798479,0.117327,1.008300,1.219422,-1.017542,0.141265,0.751045
171,1.414121,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,0.799847,0.135465,0.989973,1.218647,-1.003715,0.152968,0.752548


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    413.460
163    497.019
164    476.478
165    421.395
166    413.641
167    338.033
168    431.028
169    305.105
170    402.406
171    419.419
172    336.054
173    568.395
174    541.488
175    537.851
176    526.265
177    559.293
178    466.646
179    416.049
180    501.489
181    460.699
182    445.583
183    509.271
184    532.602
185    500.905
186    588.495
187    589.349
188    543.982
189    614.074
190    557.419
191    412.582
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [14]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3081299489, 2761246306, 1333372428, 4293897367, 1627601205, 3519704158, 4056907068, 2581267171, 2708137086, 96552343, 2267660508, 1657597269, 1236305011, 1157679008, 1190657448, 1326359214, 152701572, 1636995256, 2044370783, 3010639691, 823894793, 1280381566, 1587502250, 3428340600, 3880667578]


Step: 0 ___________________________________________
val_loss: 3370.84521484375
winner_seed: 3081299489


Step: 1 ___________________________________________
val_loss: 3305.88427734375
winner_seed: 2761246306


Step: 2 ___________________________________________
val_loss: 3412.970947265625


Step: 3 ___________________________________________
val_loss: 3890.592041015625


Step: 4 ___________________________________________
val_loss: 3070.677978515625
winner_seed: 1627601205


Step: 5 ___________________________________________
val_loss: 2377.81982421875
winner_seed: 3519704158


Step: 6 ___________________________________________
val_loss: 3547.64697265625


Step: 7 ___________________________

2023-10-14 17:02:44.465648: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 3253.5224609375


Step: 16 ___________________________________________
val_loss: 3446.580322265625


Step: 17 ___________________________________________
val_loss: 2988.728515625


Step: 18 ___________________________________________
val_loss: 2534.84375


Step: 19 ___________________________________________
val_loss: 2987.6923828125


Step: 20 ___________________________________________
val_loss: 2987.1181640625


Step: 21 ___________________________________________
val_loss: 2591.903564453125


Step: 22 ___________________________________________
val_loss: 3535.4287109375


Step: 23 ___________________________________________


2023-10-14 17:09:44.677615: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 2793.462158203125


Step: 24 ___________________________________________
val_loss: 3295.8779296875


final_seed: 3519704158


In [20]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 291879.1250 - val_loss: 270782.0312
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 231966.1719 - val_loss: 184697.4062
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 209419.1562 - val_loss: 117732.9062
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 124370.2656 - val_loss: 114278.5938
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 121176.9453 - val_loss: 84723.2578
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 92910.6875 - val_loss: 76481.1094
Epoch 7/10000
5/5 [==============================] - 0s 6ms/step - loss: 90086.2031 - val_loss: 74259.4219
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 79614.5625 - val_loss: 70755.6406
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 72225.5312 - val_loss: 64859.6758
Epoch 10/10000
5/5 [=======

5/5 [==============================] - 0s 6ms/step - loss: 10383.7168 - val_loss: 6483.4688
Epoch 78/10000
5/5 [==============================] - 0s 6ms/step - loss: 7527.6167 - val_loss: 6105.0347
Epoch 79/10000
5/5 [==============================] - 0s 6ms/step - loss: 5346.3706 - val_loss: 3615.3635
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 5340.8291 - val_loss: 3393.0864
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 4914.4390 - val_loss: 7131.9058
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 7749.7041 - val_loss: 5334.5186
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 6708.2954 - val_loss: 4884.8184
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 5894.7397 - val_loss: 7994.3447
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 5318.0132 - val_loss: 3997.1926
Epoch 86/10000
5/5 [==============================] - 0s 5ms

Epoch 154/10000
5/5 [==============================] - 0s 5ms/step - loss: 6364.1240 - val_loss: 6306.2349
Epoch 155/10000
5/5 [==============================] - 0s 5ms/step - loss: 6130.9429 - val_loss: 5369.7212
Epoch 156/10000
5/5 [==============================] - 0s 5ms/step - loss: 6015.1895 - val_loss: 5137.0029
Epoch 157/10000
5/5 [==============================] - 0s 6ms/step - loss: 5602.5122 - val_loss: 4951.0425
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 4632.7451 - val_loss: 5305.5439
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 5792.9565 - val_loss: 7286.1274
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 6061.3408 - val_loss: 8645.6797
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 7151.7407 - val_loss: 6447.8936
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 6697.8931 - val_loss: 10219.7832
Epoch 163/10000
5/5 [===============

5/5 [==============================] - 0s 6ms/step - loss: 4012.2712 - val_loss: 4695.9829
Epoch 231/10000
5/5 [==============================] - 0s 6ms/step - loss: 3534.6567 - val_loss: 3724.2778
Epoch 232/10000
5/5 [==============================] - 0s 5ms/step - loss: 3113.9648 - val_loss: 3891.3159
Epoch 233/10000
5/5 [==============================] - 0s 5ms/step - loss: 3217.7012 - val_loss: 3585.6177
Epoch 234/10000
5/5 [==============================] - 0s 6ms/step - loss: 3451.3862 - val_loss: 4399.7866
Epoch 235/10000
5/5 [==============================] - 0s 5ms/step - loss: 3856.0295 - val_loss: 4570.9697
Epoch 236/10000
5/5 [==============================] - 0s 6ms/step - loss: 5115.0098 - val_loss: 5893.8154
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 5539.6191 - val_loss: 4728.7144
Epoch 238/10000
5/5 [==============================] - 0s 6ms/step - loss: 5023.6177 - val_loss: 7462.7769
Epoch 239/10000
5/5 [==============================] 

5/5 [==============================] - 0s 7ms/step - loss: 4717.9155 - val_loss: 4379.7993
Epoch 307/10000
5/5 [==============================] - 0s 7ms/step - loss: 5418.6294 - val_loss: 8144.3130
Epoch 308/10000
5/5 [==============================] - 0s 5ms/step - loss: 7970.9556 - val_loss: 6168.2437
Epoch 309/10000
5/5 [==============================] - 0s 6ms/step - loss: 7278.2021 - val_loss: 6983.7144
Epoch 310/10000
5/5 [==============================] - 0s 6ms/step - loss: 5454.9478 - val_loss: 5701.6587
Epoch 311/10000
5/5 [==============================] - 0s 6ms/step - loss: 4765.2290 - val_loss: 5520.0073
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 4883.7769 - val_loss: 5101.7271
Epoch 313/10000
5/5 [==============================] - 0s 5ms/step - loss: 4550.9214 - val_loss: 7490.6411
Epoch 314/10000
5/5 [==============================] - 0s 6ms/step - loss: 4991.9336 - val_loss: 5727.4385
Epoch 315/10000
5/5 [==============================] 

...

5/5 [==============================] - 0s 6ms/step - loss: 3450.4983 - val_loss: 4478.1953
Epoch 915/10000
5/5 [==============================] - 0s 7ms/step - loss: 3946.7532 - val_loss: 9313.4531
Epoch 916/10000
5/5 [==============================] - 0s 5ms/step - loss: 4494.2070 - val_loss: 4883.1987
Epoch 917/10000
5/5 [==============================] - 0s 6ms/step - loss: 4328.8232 - val_loss: 4785.8364
Epoch 918/10000
5/5 [==============================] - 0s 5ms/step - loss: 4103.3149 - val_loss: 5160.8184
Epoch 919/10000
5/5 [==============================] - 0s 5ms/step - loss: 3811.3928 - val_loss: 6478.5210
Epoch 920/10000
5/5 [==============================] - 0s 5ms/step - loss: 4373.6372 - val_loss: 5713.0400
Epoch 921/10000
5/5 [==============================] - 0s 5ms/step - loss: 3949.1084 - val_loss: 6038.7036
Epoch 922/10000
5/5 [==============================] - 0s 5ms/step - loss: 3390.4072 - val_loss: 7243.3252
Epoch 923/10000
5/5 [==============================] 

5/5 [==============================] - 0s 4ms/step - loss: 3369.1404 - val_loss: 5072.3262
Epoch 991/10000
5/5 [==============================] - 0s 4ms/step - loss: 3347.2029 - val_loss: 5446.8740
Epoch 992/10000
5/5 [==============================] - 0s 5ms/step - loss: 3382.3772 - val_loss: 5131.7812
Epoch 993/10000
5/5 [==============================] - 0s 5ms/step - loss: 3458.4353 - val_loss: 8208.6484
Epoch 994/10000
5/5 [==============================] - 0s 5ms/step - loss: 3662.4319 - val_loss: 7196.5991
Epoch 995/10000
5/5 [==============================] - 0s 5ms/step - loss: 2918.6274 - val_loss: 8184.9512
Epoch 996/10000
5/5 [==============================] - 0s 5ms/step - loss: 3620.2712 - val_loss: 6801.6445
Epoch 997/10000
5/5 [==============================] - 0s 5ms/step - loss: 3163.9038 - val_loss: 6570.4067
Epoch 998/10000
5/5 [==============================] - 0s 5ms/step - loss: 3128.1541 - val_loss: 5178.2134
Epoch 999/10000
5/5 [==============================] 

5/5 [==============================] - 0s 5ms/step - loss: 5553.0977 - val_loss: 7770.9448
Epoch 1067/10000
5/5 [==============================] - 0s 5ms/step - loss: 5091.3652 - val_loss: 4311.6650
Epoch 1068/10000
5/5 [==============================] - 0s 5ms/step - loss: 3905.2075 - val_loss: 5615.0444
Epoch 1069/10000
5/5 [==============================] - 0s 5ms/step - loss: 4269.8135 - val_loss: 6571.4907
Epoch 1070/10000
5/5 [==============================] - 0s 5ms/step - loss: 4115.8057 - val_loss: 4307.0356
Epoch 1071/10000
5/5 [==============================] - 0s 5ms/step - loss: 3644.0693 - val_loss: 6363.1104
Epoch 1072/10000
5/5 [==============================] - 0s 5ms/step - loss: 5139.7231 - val_loss: 6496.1191
Epoch 1073/10000
5/5 [==============================] - 0s 5ms/step - loss: 4682.1812 - val_loss: 10444.8066
Epoch 1074/10000
5/5 [==============================] - 0s 5ms/step - loss: 5799.5854 - val_loss: 5417.5669
Epoch 1075/10000
5/5 [======================

Epoch 1142/10000
5/5 [==============================] - 0s 5ms/step - loss: 4961.5801 - val_loss: 5769.2026
Epoch 1143/10000
5/5 [==============================] - 0s 5ms/step - loss: 3620.0208 - val_loss: 6554.9839
Epoch 1144/10000
5/5 [==============================] - 0s 6ms/step - loss: 4531.3984 - val_loss: 10912.4932
Epoch 1145/10000
5/5 [==============================] - 0s 5ms/step - loss: 6420.7939 - val_loss: 7784.1738
Epoch 1146/10000
5/5 [==============================] - 0s 6ms/step - loss: 4958.8301 - val_loss: 13439.9014
Epoch 1147/10000
5/5 [==============================] - 0s 6ms/step - loss: 6114.8398 - val_loss: 8812.9619
Epoch 1148/10000
5/5 [==============================] - 0s 5ms/step - loss: 6192.6094 - val_loss: 8095.7178
Epoch 1149/10000
5/5 [==============================] - 0s 5ms/step - loss: 5324.2788 - val_loss: 4946.2993
Epoch 1150/10000
5/5 [==============================] - 0s 5ms/step - loss: 4012.6301 - val_loss: 6962.8599
Epoch 1151/10000
5/5 [====

Epoch 1218/10000
5/5 [==============================] - 0s 5ms/step - loss: 4233.9214 - val_loss: 4117.9385
Epoch 1219/10000
5/5 [==============================] - 0s 5ms/step - loss: 4097.2192 - val_loss: 4214.1299
Epoch 1220/10000
5/5 [==============================] - 0s 5ms/step - loss: 4115.5376 - val_loss: 6793.2822
Epoch 1221/10000
5/5 [==============================] - 0s 5ms/step - loss: 6095.6367 - val_loss: 7231.1553
Epoch 1222/10000
5/5 [==============================] - 0s 5ms/step - loss: 6480.8462 - val_loss: 6342.2510
Epoch 1223/10000
5/5 [==============================] - 0s 5ms/step - loss: 5182.1489 - val_loss: 5592.7407
Epoch 1224/10000
5/5 [==============================] - 0s 5ms/step - loss: 4527.3906 - val_loss: 6547.6226
Epoch 1225/10000
5/5 [==============================] - 0s 5ms/step - loss: 4429.6274 - val_loss: 6638.0947
Epoch 1226/10000
5/5 [==============================] - 0s 5ms/step - loss: 4382.2749 - val_loss: 6825.2520
Epoch 1227/10000
5/5 [======

In [21]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [22]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,420.905212,420.905212,420.905212,420.905212,420.905212,420.905212,531.633057,531.633057,531.633057,531.633057,531.633057,420.905212,531.633057,603.826294,531.633057,531.633057,531.633057,603.789612,603.826355,603.826355,603.826355,603.826355,603.826355,603.826355,603.826355,603.826355,603.823914,603.826172,531.633057,531.633057
Target,413.46,497.019,476.478,421.395,413.641,338.033,431.028,305.105,402.406,419.419,336.054,568.395,541.488,537.851,526.265,559.293,466.646,416.049,501.489,460.699,445.583,509.271,532.602,500.905,588.495,589.349,543.982,614.074,557.419,412.582
Error,7.445221,76.1138,55.572784,0.489777,7.264221,82.872223,100.605042,226.528046,129.227051,112.21405,195.579071,147.489807,9.854919,65.975281,5.368042,27.659973,64.987061,187.740601,102.337341,143.12735,158.243347,94.555359,71.224365,102.921356,15.33136,14.477356,59.841919,10.247803,25.78595,119.051056


In [23]:
display(mae)
display(mape)

80.671036

0.18851952

In [24]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [25]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[5604.502]] - Target[5022.433000000001]| =  Error: [[582.06885]]; MAPE:[[0.1158938]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[6957.1055]] - Target[5998.141]| =  Error: [[958.96436]]; MAPE:[[0.15987693]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[3478.5688]] - Target[3305.901]| =  Error: [[172.66797]]; MAPE:[[0.05223023]]


[array([[582.06885]], dtype=float32),
 array([[958.96436]], dtype=float32),
 array([[172.66797]], dtype=float32)]

571.2337

0.10933366